This is an example of merging precipitation data with the PFAS file. The precipitation data is for Orange County, 2010-2019 and was accessed from the NOAA Climate Data Online Search (https://www.ncdc.noaa.gov/cdo-web/search?datasetid=GHCND)

In [215]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import math

def deg2rad(deg):
    return(deg*(math.pi/180.))
def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2):
    R=6371.
    dlat=deg2rad(lat2-lat1)
    dlon=deg2rad(lon2-lon1)
    a=math.sin(dlat/2.)*math.sin(dlat/2.) + math.cos(deg2rad(lat1)) *\
    math.cos(deg2rad(lat2)) *math.sin(dlon/2.)*math.sin(dlon/2.)

    b = 2. * math.atan2(math.sqrt(a),math.sqrt(1-a))
    d=R*b
    return(d)

prec=pd.read_csv("precip_2010_2019.csv")[['STATION','LATITUDE','LONGITUDE','ELEVATION','DATE','PRCP','TAVG','TMAX','TMIN']]
#NOTE: The dates should be formatted as datetime objects!! (It makes it much easier to match)
prec['DATE']=pd.to_datetime(prec['DATE'])
dat=pd.read_excel("SAR-Imperial_537Data_AsOf08-08-2019.xlsx",sheet_name='All')
location_data=pd.read_excel("SAR-Imperial_537Data_AsOf08-08-2019.xlsx",sheet_name='SAR-IMPERIAL-01 Location')
dat['latitude']=float(location_data['Latitude_WGS84'])
dat['longitude']=float(location_data['Longitude_WGS84'])

In [159]:
#Function to take precipitation data and match it with the location of the PFAS data
def match_prec_pfas(prec_table,pfas_table,
                    prec_stat_name='STATION',pfas_stat_name='Station Name',
                   prec_lat_name='LATITUDE',prec_lon_name='LONGITUDE',
                   pfas_lat_name='latitude',pfas_lon_name='longitude'):
    #Find the closest precip station to the PFAS station
    #Create a unique list of precipitation monitoring stations
    prec_stations=prec_table[[prec_stat_name,prec_lat_name,prec_lon_name]].drop_duplicates()
    #Create a unique list of PFAS sources
    pfas_stations=pfas_table[[pfas_stat_name,pfas_lat_name,pfas_lon_name]].drop_duplicates()
    #Make table of distances
    comb_dist=pd.DataFrame()
    for s in pfas_stations[pfas_stat_name]:
        #Get the lat/lon coords for the station
        pfas_sub=pfas_stations[pfas_stations[pfas_stat_name]==s]
        pfas_lat=float(pfas_sub[pfas_lat_name])
        pfas_lon=float(pfas_sub[pfas_lon_name])
        for p in prec_stations[prec_stat_name]:
            prec_sub=prec_stations[prec_stations[prec_stat_name]==p]
            prec_lat=float(prec_sub[prec_lat_name])
            prec_lon=float(prec_sub[prec_lon_name])
            dict_store={}
            dict_store['pfas_s']=s
            dict_store['prec_s']=p
            #Calculate the distance between the precip and pfas locations
            dict_store['dist_betw']=getDistanceFromLatLonInKm(pfas_lat,pfas_lon,prec_lat,prec_lon)
            comb_dist=comb_dist.append(dict_store,ignore_index=True)
    #Find the min distance
    stat_match=pd.DataFrame(columns=['pfas_s','prec_s','dist_betw'])
    for s in pfas_stations[pfas_stat_name]:
        comb_sub=comb_dist[comb_dist['pfas_s']==s]
        #Find the minimum distance between pfas and precip for each pfas location
        min_dist=comb_sub.loc[comb_sub['dist_betw'].idxmin()]
        stat_match=stat_match.append(min_dist)
    return(stat_match)


In [148]:
min_dist=match_prec_pfas(prec,dat)

In [222]:
def merge_prec_pfas_df(prec_table,pfas_table,match_table,left_mergename,right_mergename,
                      prec_station_name='STATION',pfas_station_name='Station Name'):
    merged_table=pd.DataFrame()
    for w in pfas_table[pfas_station_name].unique():
        pfas_sub=pfas_table[pfas_table[pfas_station_name].str.match(w)]
        match_sub=match_table[match_table['pfas_s'].str.match(w)]
        prec_sub=prec_table[prec_table[prec_station_name].str.match(match_sub['prec_s'].item())]
        merged_dat=pd.merge(pfas_sub,prec_sub,left_on=left_mergename,right_on=right_mergename)
        merged_table=merged_table.append(merged_dat)
    return(merged_table)

In [89]:
#Merge the data by the date?
prec_tomerge=prec[prec['STATION'].str.match(min_dist['prec_s'].item())]
dat_merged=pd.merge(prec_tomerge,dat,left_on='DATE',right_on='Sample Date')

In [149]:
dat_sort=dat_merged.sort_values(by=['DATE'])
dat_sort.to_csv('SAR-Imperial_merged_precip.csv')

In [216]:
#Another dataset
new_dat=pd.read_csv('gama_pf_orange.csv')
new_dat['DATE']=pd.to_datetime(new_dat['DATE'])

In [217]:
dat_match=match_prec_pfas(prec,new_dat,pfas_stat_name='WELL ID',pfas_lat_name='APPROXIMATE LATITUDE',pfas_lon_name='APPROXIMATE LONGITUDE')

In [244]:
dat_match

,pfas_s,prec_s,dist_betw
5,3010027-023,US1CAOR0042,5.070037
101,3010092-077,US1CAOR0031,4.827372
156,3010082-005,US1CAOR0031,2.302718
170,3010068-001,US1CAOR0042,5.070037
266,3010082-007,US1CAOR0031,2.302718
284,3010001-096,USC00040192,1.018049
335,3010027-008,US1CAOR0042,5.618740
394,3010001-035,USC00040192,1.018049
493,3010037-001,US1CAOR0072,0.045659
504,3010037-003,USC00040192,1.018049


In [225]:
oc_merged=merge_prec_pfas_df(prec,new_dat,dat_match,'DATE','DATE',pfas_station_name='WELL ID')
oc_sort=oc_merged.sort_values(by='DATE')

In [246]:
oc_sort

,WELL ID,RESULTS,CHEMICAL,DATE,UNITS,QUALIFER,RL,APPROXIMATE LATITUDE,APPROXIMATE LONGITUDE,WELL TYPE,...,OTHER NAMES,YEAR,STATION,LATITUDE,LONGITUDE,ELEVATION,PRCP,TAVG,TMAX,TMIN
9,L10003191714-LEACHATE,NaN,PFUNDCA,2017-08-29,NaN,ND,50,33.712066,-117.708494,MONITORING,...,LEACHATE,2017,USC00044303,33.7200,-117.7230,164.6,0.00,NaN,99.0,66.0
7,L10003191714-BC-16,NaN,PFDOA,2017-08-29,NaN,ND,1,33.708882,-117.709872,MONITORING,...,BC-16,2017,USC00044303,33.7200,-117.7230,164.6,0.00,NaN,99.0,66.0
6,L10003191714-BC-16,0.0000,PFOA,2017-08-29,UG/L,ND,1,33.708882,-117.709872,MONITORING,...,BC-16,2017,USC00044303,33.7200,-117.7230,164.6,0.00,NaN,99.0,66.0
5,L10003191714-BC-16,NaN,PFHPA,2017-08-29,NaN,ND,1,33.708882,-117.709872,MONITORING,...,BC-16,2017,USC00044303,33.7200,-117.7230,164.6,0.00,NaN,99.0,66.0
4,L10003191714-BC-16,NaN,PFHA,2017-08-29,NaN,ND,1,33.708882,-117.709872,MONITORING,...,BC-16,2017,USC00044303,33.7200,-117.7230,164.6,0.00,NaN,99.0,66.0
3,L10003191714-BC-16,NaN,PFNDCA,2017-08-29,NaN,ND,1,33.708882,-117.709872,MONITORING,...,BC-16,2017,USC00044303,33.7200,-117.7230,164.6,0.00,NaN,99.0,66.0
1,L10003191714-BC-16,0.0000,PFOSA,2017-08-29,UG/L,ND,1,33.708882,-117.709872,MONITORING,...,BC-16,2017,USC00044303,33.7200,-117.7230,164.6,0.00,NaN,99.0,66.0
0,L10003191714-BC-16,NaN,PFUNDCA,2017-08-29,NaN,ND,1,33.708882,-117.709872,MONITORING,...,BC-16,2017,USC00044303,33.7200,-117.7230,164.6,0.00,NaN,99.0,66.0
9,L10003191714-BC-1A,0.0000,PFOA,2017-08-29,UG/L,ND,50,33.711225,-117.708767,MONITORING,...,BC-1A,2017,USC00044303,33.7200,-117.7230,164.6,0.00,NaN,99.0,66.0
8,L10003191714-BC-16,NaN,PFTRIDA,2017-08-29,NaN,ND,1,33.708882,-117.709872,MONITORING,...,BC-16,2017,USC00044303,33.7200,-117.7230,164.6,0.00,NaN,99.0,66.0


In [228]:
oc_sort.to_csv('oc_merged.csv')

In [241]:
oc_sub=oc_sort[['DATE','WELL ID','STATION','CHEMICAL','RESULTS','PRCP']].dropna()


In [243]:
oc_sub.to_csv('oc_filtered.csv')